# Notes

Emm I read the pdf and it's a bit hard to split this project into tasks tbh. From what I understood the goal, in short, is to achieve the best possible accuracy with least amount of features used. Whereas there is some reward for higher accuracy and penalty for number of features used so we need to find a proper balance. 5 different approaches should be used. It is a binary classification task. So I think we could:

1. Find some best candidates methods for binary classification tasks with numeric features.
2. One approach would be to iterate over the number of features starting from 1 (xd) and find the number that achieves the best score for each of the chosen methods
3. We could also do some feature selection methods and try to find the best number of features that way.
4. We could do some ensembling, maybe combining  models from the above

# TODO:

1. Find some good candidates for binary classification including ensembles
    1. 5 different models
2. Write a pipeline for testing and analysis of the results
3. Write the report
4. Prepare the presentation

# Load Data

In [4]:
with open('./Dataset/x_test.txt') as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open('./Dataset/x_train.txt') as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open('./Dataset/y_train.txt') as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]



## In the X_test there are 500 features and 5000 observations, y_train contains 5000 values, X_train contains 500 features and 5000 observations

In [16]:
print('X_test datapoints:',len(X_test)," features:",len(X_test[0]))
print('X_train datapoints:',len(X_train)," features:",len(X_train[0]))
print('y_train datapoints:',len(y_train))


X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000
